In [30]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [31]:
reduced = True
dataset_to_fit = 'android_data_latest'
data_type = 'wlk'
data_fit_num = 1

dataset_source_type = 'wlk_7'
dataset_source_num = 1

In [33]:
data_source_url = '../data/dataset'
if reduced:
    data_source_url += '_reduced'

data_source = pd.read_csv(data_source_url + '/' + dataset_source_type + '/sub_{}'.format(dataset_source_num) + '.csv')
data_source = data_source.drop('Unnamed: 0', axis=1) if 'Unnamed: 0' in data_source.columns else None
data_source

,gravity.x,gravity.y,gravity.z,userAcceleration.x,userAcceleration.y,userAcceleration.z
0,-0.139767,0.989987,0.019745,0.091702,0.415881,0.093689
1,-0.152324,0.987759,0.033602,0.367412,0.004566,-0.106082
2,-0.161610,0.985336,0.054729,0.172093,-0.217483,-0.163387
3,-0.168058,0.982470,0.080680,0.004957,-0.237566,-0.019080
4,-0.174613,0.978705,0.107922,-0.040292,-0.241461,0.015201
...,...,...,...,...,...,...
5434,0.268549,0.726621,0.632380,-0.690515,0.160006,-0.301676
5435,0.207323,0.679753,0.703529,-0.628969,0.368785,-0.986305
5436,0.156434,0.649222,0.744338,-0.323685,0.392786,-0.913802
5437,0.128106,0.622337,0.772195,-0.129754,0.223900,-0.717584


In [36]:
data_to_fit_url = '../data/' + dataset_to_fit
if reduced:
    data_to_fit_url += '_reduced'

if dataset_to_fit == 'android_data_latest':
    data_to_fit = pd.read_csv(data_to_fit_url + '/' + data_type + '/{}{}'.format(data_type, data_fit_num) + '-SensorData.csv')
    data_to_fit = data_to_fit.drop('Unnamed: 0', axis=1) if 'Unnamed: 0' in data_to_fit.columns else None

data_to_fit

,Gravity X,Gravity Y,Gravity Z,User Acceleration X,User Acceleration Y,User Acceleration Z
0,3.159663,-9.276977,-0.353036,-0.049377,0.006249,-0.018233
1,3.159663,-9.276977,-0.353036,-0.049377,0.006249,-0.018233
2,3.154015,-9.278942,-0.351873,-0.058250,-0.017869,-0.075712
3,3.154015,-9.278942,-0.351873,-0.058250,-0.017869,-0.075712
4,3.154015,-9.278942,-0.351873,-0.058250,-0.017869,-0.075712
...,...,...,...,...,...,...
1528,2.671978,-9.414862,0.625539,-0.000979,0.020016,-0.159608
1529,2.671978,-9.414862,0.625539,-0.000979,0.020016,-0.159608
1530,2.671978,-9.414862,0.625539,-0.000979,0.020016,-0.159608
1531,2.671978,-9.414862,0.625539,-0.000979,0.020016,-0.159608


In [37]:
if len(data_source) >= len(data_to_fit):
    data_source = data_source[:len(data_to_fit)]
else:
    data_to_fit = data_to_fit[:len(data_source)]

print('Data source shape: {}'.format(data_source.shape))
print('Data to fit shape: {}'.format(data_to_fit.shape))

Data source shape: (1533, 6)
Data to fit shape: (1533, 6)


In [38]:
# find scaling factor of each column wuth linear regression (fit android data to original data)
def calculate_scaling_factors(df, df_o):
    scaling_factors = {}

    for i in range(len(df.columns)):
        X = df.iloc[:, i].values.reshape(-1, 1)
        y = df_o.iloc[:, i].values.reshape(-1, 1)

        model = LinearRegression(fit_intercept=False)
        model.fit(X, y)
        scaling_factors[df.columns[i]] = model.coef_[0][0]

    return scaling_factors

In [39]:
# predict
scaling_factors = calculate_scaling_factors(data_to_fit, data_source)
scaling_factors

{'Gravity X': -0.0959586547427039,
 'Gravity Y': -0.09569122769633755,
 'Gravity Z': 0.09948208173715353,
 'User Acceleration X': 0.12905120595974687,
 'User Acceleration Y': 0.012367038275536486,
 'User Acceleration Z': 0.048466978127596386}

In [40]:
# Write to Constants File
constant_file_url = '../constants.txt'
if reduced:
    constant_file_url = '../constants_reduced.txt'

with open(constant_file_url, 'a') as f:
    f.write(str(scaling_factors) + '\n')

In [43]:
# opening scaling factor file to read the dictionary
factors = []
with open(constant_file_url, 'r') as f:
    lines = f.readlines()
    for line in lines:
        factors.append(eval(line))

factors

[{'Gravity X': -0.029648750139525837,
  'Gravity Y': -0.2301603051826162,
  'Gravity Z': -0.15008394265004601,
  'User Acceleration X': 0.007038958443613083,
  'User Acceleration Y': 0.005498103809718967,
  'User Acceleration Z': 0.05049859283824429},
 {'Gravity X': -0.0959586547427039,
  'Gravity Y': -0.09569122769633755,
  'Gravity Z': 0.09948208173715353,
  'User Acceleration X': 0.12905120595974687,
  'User Acceleration Y': 0.012367038275536486,
  'User Acceleration Z': 0.048466978127596386}]